# Question 1.g

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

torch.set_printoptions(precision=10)
torch.__version__

## Load train and test dataset

In [ ]:
class OccupancyDataset(Dataset):
    """ Occupancy detection dataset"""
    
    def __init__(self, filename):
        self.dataset = np.loadtxt(filename, skiprows=1, dtype = object, delimiter=',')
        
        self.data = np.array(self.dataset[:,2:7], dtype = float)
        self.data = (self.data - self.data.mean())/self.data.std()
        self.data = torch.from_numpy(self.data).float()
        
        self.target = np.array(self.dataset[:,7], dtype = float)
        self.target = torch.from_numpy(self.target).float()
        
    def __len__(self):
        return self.data.size()[0]
    
    def __getitem__(self, idx):
        return (self.data[idx, :], self.target[idx])

    
# Setup datasets and loaders
train_dataset = OccupancyDataset('train_data.txt')
test_dataset = OccupancyDataset('test_data.txt')

train_batch_size = 100
test_batch_size = 100

train_loader = DataLoader(train_dataset, batch_size = train_batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size= test_batch_size, shuffle = False)

## Create network class

In [3]:
class Net(nn.Module):
    """ Neural network with 2 hidden layers and sigmoid activations """
    
    def __init__(self, h1, h2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2,1)
        
    def forward(self, x):
        #print(x)
        x = F.sigmoid(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return F.sigmoid(self.fc3(x))
    

## Setting variables

In [4]:
epochs = 2000
lr = 0.001
criterion = nn.BCELoss()

## Train function

In [5]:
def train(epoch):
    # Train the network
    running_loss = 0
    count = 0
    for i, data in enumerate(train_loader, 0):
        optimizer.zero_grad()
        inputs, target = data
        inputs, target = Variable(inputs), Variable(target.float())
        output = net(inputs).squeeze()
        loss =  criterion(output, target)     
        loss.backward()
        optimizer.step()
        running_loss += loss.data[0]
        count +=1

    return running_loss/count
    
    
def get_acc(loader, dataset):
    # Get accuracy (%) for train/test
    correct = 0
    total = 0
    for i, data in enumerate(loader, 0):
        inputs, target = data
        inputs, target = Variable(inputs), Variable(target.float())
        output = net(inputs).squeeze()
        output = output>0.5
        correct += (target.data.cpu().numpy() == output.data.cpu().numpy()).sum()
        total += target.size(0)
    return 100*int(correct)/int(total)
    

In [8]:
hidden_units_1 = [5,10,20]
hidden_units_2 = [1,2,5,10,20,40]

for h1 in hidden_units_1:
    for h2 in hidden_units_2:
        net = Net(h1, h2)
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
        errors = []
        print('\nTraining neural network with h1 = {0} and h2 = {1}'.format(h1, h2))
        # Run epochs and store necessary metrics
        for i in range(epochs):
            loss = train(i)
            train_acc = get_acc(train_loader, 'Train')
            errors.append(100-train_acc)
            test_acc = get_acc(test_loader, 'Test')
            #if i%100==0:
                #print('Loss after epoch {1}: {0}'.format(loss, i))
                #print('Train Accuracy: {0}'.format(train_acc))
                #print('Test Accuracy: {0}'.format(test_acc))

                # Stop training based on below condition
            if i>200 and errors[i-50]-errors[i] < 1e-2:
                break

        print('Final test accuracy for h1={1} and h2={2} : {0}'.format(test_acc, h1, h2))
            



Training neural network with h1 = 5 and h2 = 1
Final test accuracy for h1=5 and h2=1 : 78.98892534864643

Training neural network with h1 = 5 and h2 = 2
Final test accuracy for h1=5 and h2=2 : 99.12838392124692

Training neural network with h1 = 5 and h2 = 5
Final test accuracy for h1=5 and h2=5 : 98.63617719442166

Training neural network with h1 = 5 and h2 = 10
Final test accuracy for h1=5 and h2=10 : 97.80557834290401

Training neural network with h1 = 5 and h2 = 20
Final test accuracy for h1=5 and h2=20 : 98.83100902378999

Training neural network with h1 = 5 and h2 = 40
Final test accuracy for h1=5 and h2=40 : 98.20549630844955

Training neural network with h1 = 10 and h2 = 1
Final test accuracy for h1=10 and h2=1 : 98.21575061525841

Training neural network with h1 = 10 and h2 = 2
Final test accuracy for h1=10 and h2=2 : 98.76948318293684

Training neural network with h1 = 10 and h2 = 5
Final test accuracy for h1=10 and h2=5 : 98.18498769483183

Training neural network with h1 =

### Observations

- The highest test accuracy amongst the above is 99.13 % for h1 = 5 and h2 = 2
- This accuracy is higher than that obtained for SGD using only one hidden layer.